# Solutions for Lesson 3: Main Project

Don't look at this notebook until you have attempted to solve the problems yourself.

<br><br><br><br><br>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn.datasets
import torch
from torch import nn

In [2]:
hls4ml_lhc_jets_hlf = sklearn.datasets.fetch_openml("hls4ml_lhc_jets_hlf")

features, targets = hls4ml_lhc_jets_hlf["data"], hls4ml_lhc_jets_hlf["target"]

In [12]:
features_tensor = torch.tensor(features.values).float()
targets_tensor = torch.tensor(targets.cat.codes.values)